In [32]:
import sqlite3
import pandas as pd

In [33]:
csv_data = pd.read_csv("output_norbert.csv")
pd.to_datetime(csv_data['date_key'])
pd.options.display.min_rows = 40
pd.options.display.max_rows = 40
pd.options.display.max_columns = 99
pd.options.display.max_colwidth = 30
pd.options.display.width = 9999

In [34]:
con = sqlite3.connect(":memory:")

In [35]:
csv_data.to_sql("users", con, if_exists="replace")

43167

In [36]:
query1 = """

WITH table1_temp AS (
SELECT
account_id,
date_key,
revenue,
LEAD(date_key, 1) OVER (PARTITION BY account_id ORDER BY date_key) as lead_1_month,
LEAD(date_key, 3) OVER (PARTITION BY account_id ORDER BY date_key) as lead_3_month,
LEAD(date_key, 6) OVER (PARTITION BY account_id ORDER BY date_key) as lead_6_month,
LEAD(date_key, 12) OVER (PARTITION BY account_id ORDER BY date_key) as lead_12_month
FROM users
WHERE 1=1
AND account_id = '5edf63c9-a4de-4592-b70d-8fa137c00f88'
),

table2_temp AS (
SELECT
account_id,
date_key,
revenue,
ROUND((JULIANDAY(date_key) - JULIANDAY(lead_1_month))/(365/12),0) as lead_1_mo_diff,
ROUND((JULIANDAY(date_key) - JULIANDAY(lead_3_month))/(365/12),0) as lead_3_mo_diff,
ROUND((JULIANDAY(date_key) - JULIANDAY(lead_6_month))/(365/12),0) as lead_6_mo_diff,
ROUND((JULIANDAY(date_key) - JULIANDAY(lead_12_month))/(365/12),0) as lead_12_mo_diff
FROM table1_temp
),

table3_temp AS (
SELECT
date_key,
COUNT(account_id) AS c_cur,
COUNT(CASE WHEN lead_1_mo_diff=-1.0 THEN account_id ELSE NULL END) AS c_1m,
COUNT(CASE WHEN lead_3_mo_diff=-3.0 THEN account_id ELSE NULL END) AS c_3m,
COUNT(CASE WHEN lead_6_mo_diff=-6.0 THEN account_id ELSE NULL END) AS c_6m,
COUNT(CASE WHEN lead_12_mo_diff=-12.0 THEN account_id ELSE NULL END) AS c_12m
FROM table2_temp
GROUP BY 1
)

SELECT date_key, lead_1_month, lead_3_month, lead_6_month, lead_12_month FROM table1_temp

"""

query2 = """
SELECT * FROM users where account_id = '5edf63c9-a4de-4592-b70d-8fa137c00f88'
"""

query3 = """
WITH calendar AS (
SELECT
DISTINCT(date_key)
FROM users
ORDER BY date_key
),

table1_temp AS (
SELECT
account_id,
date_key,
revenue,
LEAD(date_key, 1) OVER (ORDER BY date_key) as lead_1_month,
IFNULL(ROUND((JULIANDAY(date_key) - JULIANDAY(LEAD(date_key, 1) OVER (ORDER BY date_key)))/(365/12),0), 0.0) as lead_1_mo_diff,
LEAD(date_key, 3) OVER (PARTITION BY account_id ORDER BY date_key) as lead_3_month,
IFNULL(ROUND((JULIANDAY(date_key) - JULIANDAY(LEAD(date_key, 3) OVER (PARTITION BY account_id ORDER BY date_key)))/(365/12),0), 0.0) as lead_3_mo_diff,
LEAD(date_key, 6) OVER (PARTITION BY account_id ORDER BY date_key) as lead_6_month,
IFNULL(ROUND((JULIANDAY(date_key) - JULIANDAY(LEAD(date_key, 6) OVER (PARTITION BY account_id ORDER BY date_key)))/(365/12),0), 0.0) as lead_6_mo_diff,
LEAD(date_key, 12) OVER (PARTITION BY account_id ORDER BY date_key) as lead_12_month,
IFNULL(ROUND((JULIANDAY(date_key) - JULIANDAY(LEAD(date_key, 12) OVER (PARTITION BY account_id ORDER BY date_key)))/(365/12),0), 0.0) as lead_12_mo_diff
FROM users
WHERE 1=1
AND account_id IN ('5edf63c9-a4de-4592-b70d-8fa137c00f88', 'b41abbf3-ca43-48cd-8e8d-1aad0729bae6')
AND date_key BETWEEN '2019-01-01' AND '2019-12-31'
),

table2_temp AS (
SELECT
account_id,
date_key,
revenue,
lead_1_month,
lead_1_mo_diff,
SUM(CASE WHEN lead_1_mo_diff = -1.0 THEN 1 ELSE 0 END) as c_1m,
lead_3_month,
lead_3_mo_diff,
SUM(CASE WHEN lead_3_mo_diff = -3.0 THEN 1 ELSE 0 END) as c_3m
--date_key,
--COUNT(account_id) AS c_cur,
--COUNT(CASE WHEN lead_1_mo_diff=-1.0 THEN account_id ELSE NULL END) AS c_1m,
--COUNT(CASE WHEN lead_3_mo_diff=-3.0 THEN account_id ELSE NULL END) AS c_3m,
--COUNT(CASE WHEN lead_6_mo_diff=-6.0 THEN account_id ELSE NULL END) AS c_6m,
--COUNT(CASE WHEN lead_12_mo_diff=-12.0 THEN account_id ELSE NULL END) AS c_12m
FROM table1_temp
GROUP BY
account_id,
date_key,
revenue,
lead_1_month,
lead_1_mo_diff,
lead_3_month,
lead_3_mo_diff
)

-- SELECT * FROM calendar


SELECT
account_id,
date_key,
revenue,
lead_1_month,
lead_1_mo_diff,
--SUM(c_1m),
lead_3_month,
lead_3_mo_diff
--SUM(c_3m)
FROM table1_temp
ORDER BY 
date_key

"""

query4 = """
WITH calendar AS (
SELECT
date_key,
LEAD(date_key, 3) OVER (ORDER BY date_key) as date_key3,
LEAD(date_key, 6) OVER (ORDER BY date_key) as date_key6,
LEAD(date_key, 12) OVER (ORDER BY date_key) as date_key12
FROM users
WHERE 1=1
AND date_key > '2019-06-01'
GROUP BY date_key
ORDER BY date_key
),

fact_table_curr AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
),

fact_table_3m AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
),

fact_table_6m AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
),

fact_table_12m AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
),

all_joined AS (
SELECT
calendar.date_key as c_date_key,
fact_table_curr.date_key as f_curr_date_key,
SUBSTR(fact_table_curr.account_id,1,3) as f_curr_acc_id,
fact_table_curr.revenue as f_curr_rev,
calendar.date_key3 as c_date_key3,
fact_table_3m.date_key as f_3m_date_key,
SUBSTR(fact_table_3m.account_id,1,3) as f_3m_acc_id,
fact_table_6m.revenue as f_6m_rev,
calendar.date_key6 as c_date_key6,
fact_table_6m.date_key as f_6m_date_key,
SUBSTR(fact_table_6m.account_id,1,3) as f_6m_acc_id,
fact_table_6m.revenue as f_6m_rev,
calendar.date_key12 as c_date_key12,
fact_table_12m.date_key as f_12m_date_key,
SUBSTR(fact_table_12m.account_id,1,3) as f_12m_acc_id,
fact_table_12m.revenue as f_12m_rev
FROM calendar
LEFT JOIN fact_table_curr ON calendar.date_key = fact_table_curr.date_key
LEFT JOIN fact_table_3m ON (fact_table_curr.account_id = fact_table_3m.account_id AND calendar.date_key3 = fact_table_3m.date_key)
LEFT JOIN fact_table_6m ON (fact_table_curr.account_id = fact_table_6m.account_id AND calendar.date_key6 = fact_table_6m.date_key)
LEFT JOIN fact_table_12m ON (fact_table_curr.account_id = fact_table_12m.account_id AND calendar.date_key12 = fact_table_12m.date_key)
)

SELECT
c_date_key,
COUNT(f_curr_acc_id) as count_acc_curr,
COUNT(f_3m_acc_id) as count_acc_3m,
COUNT(f_6m_acc_id) as count_acc_6m,
COUNT(f_12m_acc_id) as count_acc_12m
FROM all_joined
WHERE 1=1
GROUP BY
c_date_key
ORDER BY
c_date_key

"""

query4_verif = """
WITH calendar AS (
SELECT
date_key,
LEAD(date_key, 1) OVER () as date_key1,
LEAD(date_key, 3) OVER () as date_key3,
LEAD(date_key, 6) OVER () as date_key6
--LEAD(date_key, 12) OVER () as date_key12
FROM users
WHERE 1=1
AND date_key BETWEEN '2019-01-01' AND '2019-07-01'
GROUP BY date_key
ORDER BY date_key
),

fact_table1 AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
AND date_key BETWEEN '2019-01-01' AND '2019-12-01'
AND account_id IN (
    'eba9dbd2-ff64-46cb-9ab8-08ddcc99485f',
    'c51272a2-28ad-4fa8-80c1-5b7566c439a8',
    '5a316a69-9d5e-4bdc-a908-8a109ed7f4ac',
    '17516fd3-55e1-4ebb-8050-2da61969976e',
    'b2ec20fc-27d1-43cf-83a3-e0ada52d0639'
    )
),

fact_table2 AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
AND date_key BETWEEN '2019-01-01' AND '2019-12-01'
AND account_id IN (
    'eba9dbd2-ff64-46cb-9ab8-08ddcc99485f',
    'c51272a2-28ad-4fa8-80c1-5b7566c439a8',
    '5a316a69-9d5e-4bdc-a908-8a109ed7f4ac',
    '17516fd3-55e1-4ebb-8050-2da61969976e',
    'b2ec20fc-27d1-43cf-83a3-e0ada52d0639'
    )
),

all_joined AS (
SELECT
calendar.date_key as c_date_key,
fact_table1.date_key as f1_date_key,
SUBSTR(fact_table1.account_id,1,3) as f1_acc_id,
fact_table1.revenue as f1_rev,
calendar.date_key3 as c_date_key3,
fact_table2.date_key as f2_date_key,
SUBSTR(fact_table2.account_id,1,3) as f2_acc_id,
fact_table2.revenue as f2_rev
FROM calendar
LEFT JOIN fact_table1 ON calendar.date_key = fact_table1.date_key
LEFT JOIN fact_table2 ON fact_table1.account_id = fact_table2.account_id AND calendar.date_key3 = fact_table2.date_key
)

SELECT
*
FROM all_joined
WHERE 1=1


"""

query4_presto = """ 

WITH calendar AS (
SELECT
date_key,
LEAD(date_key, 3) OVER (ORDER BY date_key) as date_key3,
LEAD(date_key, 6) OVER (ORDER BY date_key) as date_key6,
LEAD(date_key, 12) OVER (ORDER BY date_key) as date_key12
FROM clv.agg_acc_rev_monthly
WHERE date_key BETWEEN '2019-01-31' AND '2019-06-30'
GROUP BY date_key
ORDER BY date_key
),

fact_table_curr AS (
SELECT
account_id,
date_key,
hr as revenue
FROM clv.agg_acc_rev_monthly
WHERE 1=1
),

fact_table_3m AS (
SELECT
account_id,
date_key,
hr as revenue
FROM clv.agg_acc_rev_monthly
WHERE 1=1
),

fact_table_6m AS (
SELECT
account_id,
date_key,
hr as revenue
FROM clv.agg_acc_rev_monthly
WHERE 1=1
),

fact_table_12m AS (
SELECT
account_id,
date_key,
hr as revenue
FROM clv.agg_acc_rev_monthly
WHERE 1=1
),

all_joined AS (
SELECT
calendar.date_key as c_date_key,
fact_table_curr.date_key as f_curr_date_key,
fact_table_curr.account_id as f_curr_acc_id,
fact_table_curr.revenue as f_curr_rev,
calendar.date_key3 as c_date_key3,
fact_table_3m.date_key as f_3m_date_key,
fact_table_3m.account_id as f_3m_acc_id,
fact_table_3m.revenue as f_3m_rev
--calendar.date_key6 as c_date_key6,
--fact_table_6m.date_key as f_6m_date_key,
--fact_table_6m.account_id as f_6m_acc_id,
--fact_table_6m.revenue as f_6m_rev,
--calendar.date_key12 as c_date_key12,
--fact_table_12m.date_key as f_12m_date_key,
--fact_table_12m.account_id as f_12m_acc_id,
--fact_table_12m.revenue as f_12m_rev
FROM calendar
LEFT JOIN fact_table_curr ON calendar.date_key = fact_table_curr.date_key
LEFT JOIN fact_table_3m ON (fact_table_curr.account_id = fact_table_3m.account_id AND calendar.date_key3 = fact_table_3m.date_key)
--LEFT JOIN fact_table_6m ON (fact_table_curr.account_id = fact_table_6m.account_id AND calendar.date_key6 = fact_table_6m.date_key)
--LEFT JOIN fact_table_12m ON (fact_table_curr.account_id = fact_table_12m.account_id AND calendar.date_key12 = fact_table_12m.date_key)
)

SELECT
c_date_key,
COUNT(f_curr_acc_id) as count_acc_curr,
COUNT(f_3m_acc_id) as count_acc_3m
FROM all_joined
WHERE 1=1
GROUP BY
c_date_key
ORDER BY
c_date_key

"""



In [37]:
with open(file='users.sql', mode='r') as f:
    query5 = f.read()

cohort_df = pd.read_sql_query(query5, con)
print(cohort_df)

cohort_df.to_sql("users_2", con, index=False, if_exists="replace")

         date_key                     account_id   revenue       source
0      2019-01-01  00a9e301-7652-48e2-9cb6-98...  6.023267  display_ads
1      2019-02-01  00a9e301-7652-48e2-9cb6-98...  2.894099  display_ads
2      2019-03-01  00a9e301-7652-48e2-9cb6-98...  7.416970  display_ads
3      2019-04-01  00a9e301-7652-48e2-9cb6-98...  7.410220  display_ads
4      2019-05-01  00a9e301-7652-48e2-9cb6-98...  0.263766  display_ads
5      2019-06-01  00a9e301-7652-48e2-9cb6-98...  9.898501  display_ads
6      2019-07-01  00a9e301-7652-48e2-9cb6-98...  0.000000  display_ads
7      2019-08-01  00a9e301-7652-48e2-9cb6-98...  0.000000  display_ads
8      2019-09-01  00a9e301-7652-48e2-9cb6-98...  1.945312  display_ads
9      2019-10-01  00a9e301-7652-48e2-9cb6-98...  4.281214  display_ads
10     2019-11-01  00a9e301-7652-48e2-9cb6-98...  9.562664  display_ads
11     2019-12-01  00a9e301-7652-48e2-9cb6-98...  3.568451  display_ads
12     2020-01-01  00a9e301-7652-48e2-9cb6-98...  0.000000  disp

24000

In [38]:
with open(file='users_2.sql', mode='r') as f:
    query6 = f.read()

users_df = pd.read_sql_query(query6, con)
print(users_df)

      date_key   a  b  c
0   2019-02-01   0  0  0
1   2019-03-01   0  0  0
2   2019-04-01   0  0  0
3   2019-05-01  19  0  0
4   2019-06-01  19  0  0
5   2019-07-01  17  0  0
6   2019-08-01  15  2  0
7   2019-09-01  25  2  0
8   2019-10-01  12  1  0
9   2019-11-01  10  2  0
10  2019-12-01  16  1  0
11  2020-01-01  18  0  0
12  2020-02-01  19  1  0
13  2020-03-01  17  0  0
14  2020-04-01  24  3  0
15  2020-05-01  17  2  0
16  2020-06-01  14  2  0
17  2020-07-01  14  2  0
18  2020-08-01  24  1  0
19  2020-09-01  22  1  0
20  2020-10-01  13  2  0
21  2020-11-01  11  2  0
22  2020-12-01  14  1  0
